In [1]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=9b8fb88f202dbcadbc532c1385c5fe497eb721c312c4903cd76231437f0f56f6
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


# Recommender Code Along
The classic recommender tutorial uses the movielens data set. It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!

Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('rec').getOrCreate()

In [3]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

[video](https://camo.githubusercontent.com/9579a3c24e3a37546ef7925d459af522d55f577ca68b0f583ca02aac6e4e0050/68747470733a2f2f75706c6f61642e77696b696d656469612e6f72672f77696b6970656469612f636f6d6d6f6e732f352f35322f436f6c6c61626f7261746976655f66696c746572696e672e676966)

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [4]:
data=spark.read.csv('/content/sample_data/movielens_ratings.csv',inferSchema=True,header=True)

In [6]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [5]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [7]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [8]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [9]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

Now let's see how the model performed!

In [10]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [11]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      1|   1.0|    28|  1.1647227|
|      0|   1.0|    22|  2.2299001|
|      5|   2.0|    22|   2.416229|
|      1|   1.0|     6|-0.96241426|
|      2|   1.0|    16|   3.578138|
|      2|   3.0|     6|   3.005766|
|      6|   1.0|     6|  1.1318676|
|      2|   1.0|     3| 0.49013335|
|      0|   1.0|     5|  1.4943954|
|      4|   1.0|     5|   2.233136|
|      1|   1.0|    19|   0.716948|
|      3|   1.0|     9|  1.1507212|
|      4|   1.0|     9| 0.32034153|
|      6|   1.0|     9|  0.6330774|
|      6|   1.0|     4|0.007113615|
|      3|   2.0|     8| 0.85784036|
|      3|   1.0|     7| 0.69899887|
|      2|   4.0|    10|   3.064523|
|      1|   3.0|    25|  1.6535751|
|      0|   1.0|    11| -1.2779042|
+-------+------+------+-----------+
only showing top 20 rows



In [12]:
evaluator=RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')
rmse=evaluator.evaluate(predictions)
print('RMSE:')
print(rmse)

RMSE:
2.0628686906662717


The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [13]:
single_user=test.filter(test['userId']==11).select(['movieId','userId'])

In [14]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     10|    11|
|     22|    11|
|     40|    11|
|     41|    11|
|     47|    11|
|     70|    11|
|     77|    11|
|     80|    11|
|     81|    11|
+-------+------+



In [15]:
reccomendations = model.transform(single_user)

In [16]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     47|    11| 5.1156373|
|     22|    11|  4.809007|
|     70|    11| 1.9249151|
|     81|    11| 1.8753259|
|     77|    11| 1.6578772|
|     41|    11|0.83230907|
|     10|    11| 0.6917889|
|      0|    11|-1.2779042|
|     80|    11|-1.3224785|
|     40|    11|-2.0610735|
+-------+------+----------+

